In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
print('Importing libraries...')
import os
#import numpy as np
#import pandas as pd 
import multiprocessing
from scipy.misc import imread, imresize
from sklearn.tree import DecisionTreeClassifier 
from sklearn.cross_validation import cross_val_score

labels = 'c0,c1,c2,c3,c4,c5,c6,c7,c8,c9'.split(',')
print('Defining data...')
def get_train():
    one_up = os.path.dirname(os.getcwd())
    labels = [i for i in os.listdir(os.path.join(one_up, 'input', 'train')) if 'c' in i]
    labels.sort()
    data = []
    
    for lab in labels:
        paths = os.listdir(os.path.join('..', 'input','train', lab))
        X = [(os.path.join(one_up, 'input', 'train', lab, i), lab) for i in paths]
        data.extend(X)
    import random
    random.shuffle(data) # since labels were sorted
    df = pd.DataFrame({'paths': [i[0] for i in data],
                       'target': [i[1] for i in data]})

    for cl in labels:
        df[cl] = df.target == cl
    df.drop('target', 1, inplace=True)
    return df

train = get_train().sample(2000)


In [ ]:
print('Importing libraries...')
import os
import numpy as np
import pandas as pd 
import multiprocessing
from scipy.misc import imread, imresize
from sklearn.tree import DecisionTreeClassifier 
from sklearn.cross_validation import cross_val_score

labels = 'c0,c1,c2,c3,c4,c5,c6,c7,c8,c9'.split(',')

print('Defining data...')
def get_train():
    one_up = os.path.dirname(os.getcwd())
    labels = [i for i in os.listdir(os.path.join(one_up, 'input', 'train')) if 'c' in i]
    labels.sort()
    data = []
    
    for lab in labels:
        paths = os.listdir(os.path.join('..', 'input','train', lab))
        X = [(os.path.join(one_up, 'input', 'train', lab, i), lab) for i in paths]
        data.extend(X)
    import random
    random.shuffle(data) # since labels were sorted
    df = pd.DataFrame({'paths': [i[0] for i in data],
                       'target': [i[1] for i in data]})

    for cl in labels:
        df[cl] = df.target == cl
    df.drop('target', 1, inplace=True)
    return df

train = get_train().sample(2000)
#print(train.shape)
#print(get_train())
print(train["paths"])

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

def getimage(x):
    return imresize(imread(x, 'L'), (100, 100)).flatten()

print('Loading training data...')
train['images'] = apply_by_multiprocessing(train.paths, getimage) 
X = np.array([i for i in train.images])

print('Training classifiers...')
classifiers = [DecisionTreeClassifier(max_depth=10) for i in labels]
targets = [train[i] for i in labels]

#print(X.shape)
#print(targets)
